In [0]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from math import radians, cos, sin, asin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#Reading Data In
cities = pd.read_csv('WDataFiles/WCities.csv')
game_cities = pd.read_csv('WDataFiles/WGameCities.csv')
tourney_compact= pd.read_csv('WDataFiles/WNCAATourneyCompactResults.csv')
tourney_detailed = pd.read_csv('WDataFiles/WNCAATourneyDetailedResults.csv')
tourney_seed = pd.read_csv('WDataFiles/WNCAATourneySeeds.csv')
tourney_slots = pd.read_csv('WDataFiles/WNCAATourneySlots.csv')
reg_seas_compact = pd.read_csv('WDataFiles/WRegularSeasonCompactResults.csv')
reg_seas_detail = pd.read_csv('WDataFiles/WRegularSeasonDetailedResults.csv')
seasons = pd.read_csv('WDataFiles/WSeasons.csv')
teams = pd.read_csv('WDataFiles/WTeams.csv')
team_spellings = pd.read_csv('WDataFiles/WTeamSpellings.csv',encoding = "ISO-8859-1")

# Mapping Team Ids to Team Names & citiesids to city names
label_map = {"TeamID":"Team","WTeamID":"WTeam","LTeamID":"LTeam"}
team_map = teams.set_index(teams['TeamID']).drop('TeamID',axis=1).to_dict()['TeamName']
game_cities['WTeamID'] = game_cities['WTeamID'].apply(lambda x: team_map[x])
game_cities['LTeamID'] = game_cities['LTeamID'].apply(lambda x: team_map[x])
game_cities.rename(columns=label_map, inplace=True)
tourney_compact['WTeamID'] = tourney_compact['WTeamID'].apply(lambda x: team_map[x])
tourney_compact['LTeamID'] = tourney_compact['LTeamID'].apply(lambda x: team_map[x])
tourney_compact.rename(columns=label_map, inplace=True)
tourney_compact = tourney_compact[(tourney_compact.Season >=2010)&(tourney_compact.Season<2018)]
tourney_detailed['WTeamID'] = tourney_detailed['WTeamID'].apply(lambda x: team_map[x])
tourney_detailed['LTeamID'] = tourney_detailed['LTeamID'].apply(lambda x: team_map[x])
tourney_detailed.rename(columns=label_map, inplace=True)
tourney_detailed = tourney_detailed[(tourney_detailed.Season >=2010)&(tourney_detailed.Season<2018)]
tourney_seed['TeamID'] = tourney_seed['TeamID'].apply(lambda x: team_map[x])
tourney_seed.rename(columns=label_map, inplace=True)
reg_seas_compact['WTeamID'] = reg_seas_compact['WTeamID'].apply(lambda x: team_map[x])
reg_seas_compact['LTeamID'] = reg_seas_compact['LTeamID'].apply(lambda x: team_map[x])
reg_seas_compact.rename(columns=label_map, inplace=True)
reg_seas_compact = reg_seas_compact[(reg_seas_compact.Season >=2010)&(reg_seas_compact.Season<2018)]
reg_seas_detail['WTeamID'] = reg_seas_detail['WTeamID'].apply(lambda x: team_map[x])
reg_seas_detail['LTeamID'] = reg_seas_detail['LTeamID'].apply(lambda x: team_map[x])
reg_seas_detail.rename(columns=label_map, inplace=True)
reg_seas_detail = reg_seas_detail[(reg_seas_detail.Season >=2010)&(reg_seas_detail.Season<2018)]
game_cities = game_cities.merge(cities, how='left')
game_cities = game_cities.drop('CityID', axis=1)

FileNotFoundError: ignored

# Producing Beat-up Metric

In [0]:
'''
Adding Lat/Long coordinates to game_cities and fixing/adding city names so merge will be successfull;
Lat Long data obtained from Simplemaps.com
'''
g_cities_c = game_cities.copy()
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Moon Township'].index.to_list(),'City'] = 'Moon'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Vestal'].index.to_list(),'City'] = 'Vestal Center'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Notre Dame'].index.to_list(),'City'] = 'South Bend'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Presbyterian'].index.to_list(),'City'] = 'Clinton'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Niagara University'].index.to_list(),'City'] = 'Lewiston'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Moorehead'].index.to_list(),'City'] = 'Moorhead'
g_cities_c.loc[g_cities_c[g_cities_c.City == 'Riverdale'].index.to_list(),'City'] = 'New York'
locations = pd.read_csv('WDataFiles/uscitiesv1.4.csv')
locations['Lat/Long'] = locations.apply(lambda row: [row['lat'], row['lng']], axis=1)
cities_ll = locations[['city','state_id','Lat/Long']]
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['South Orange','NJ',[40.7426,-74.2465]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['USAF Academy','CO',[38.9983,-104.8613]]]),ignore_index=True)
cities_ll.city = cities_ll.city.apply(lambda x: x.replace('Saint ','St. ')if (('Saint ') in x) else x)
cities_ll.city = cities_ll.city.apply(lambda x: x.replace('Mount ','Mt. ')if (('Mount ') in x) else x)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Chestnut Hill','MA',[42.3355,-71.1685]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Cancun','MX',[21.1619,-86.8515]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Laie','HI',[21.6483,-157.9226]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Playa del Carmen','MX',[20.6296,-87.0739]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Playa del Carmen','MX',[20.6296,-87.0739]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Puerto Vallarta','MX',[20.6534,-105.2253]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Lucaya','BA',[26.5096,-78.6417]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Bimini','BA',[25.7332976,-79.2729342]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Northridge','CA',[34.2381,-118.5301]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['St. Thomas','VI',[18.3381,-64.8941]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Freeport','BA',[26.53333,-78.7124]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Amherst','MA',[42.3732,-72.5199]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Fairfield','CT',[41.1408,-73.2613]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Smithfield','RI',[41.9220,-71.5495]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['University Park','PA',[40.8148,-77.8653]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Nassau','BA',[25.0480,-77.3554]]]),ignore_index=True)
cities_ll = cities_ll.append(pd.DataFrame(columns=['city','state_id','Lat/Long'],data=[['Hamden','CT',[41.3839,-72.9026]]]),ignore_index=True)
cities_ll.rename(columns={'city':'City','state_id':'State','Lat/Long':'LL'},inplace=True)
LL_games = g_cities_c.merge(cities_ll,on=['City','State'],right_index=True).sort_values(['Season','DayNum']).reset_index(drop=True)
LL_games = LL_games.assign(City_State = lambda x: x['City']+ "," + x['State'])

# Haversine function to calculate distance between two lat/long coordinates 
def haversine1(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 
    return c * r

# Creating table with variables that will be used to create beatup metric; table also contains results variable 
unique_teams = np.union1d(reg_seas_detail.WTeam.unique(), reg_seas_detail.LTeam.unique())
tuples=[]
for team in unique_teams:
    for season in reg_seas_detail.Season.unique():
        tup = (team,season)
        tuples.append(tup)        
index = pd.MultiIndex.from_tuples(tuples, names=['Team', 'Season'])
reg_seas_beatup = pd.DataFrame(index=index, columns=['AVG OT','AVG Days of Rest','Median Fouls Against','Median Travel Distance','Result'])
for index in reg_seas_beatup.index:
    team, season  = index
    team_row = reg_seas_detail[((reg_seas_detail['WTeam']==team)|(reg_seas_detail['LTeam']==team))&(reg_seas_detail['Season']==season)]
    if len(team_row) == 0:
        continue
    ot = team_row.NumOT.mean()
    rest = team_row.DayNum.diff().mean()
    tourney = tourney_detailed[tourney_detailed.Season==season]
    tourney_teams = np.union1d(tourney.WTeam.unique(), tourney.LTeam.unique())
    final16 = tourney[-15:]
    final16_teams = np.union1d(final16.WTeam.unique(), final16.LTeam.unique())
    final4 = tourney[-3:]
    final4_teams = np.union1d(final4.WTeam.unique(), final4.LTeam.unique())
    if team not in tourney_teams:
        result = 0
    elif team not in final16_teams:
        result = 1
    elif team not in final4_teams:
        result = 2
    else: 
        result = 3
    fouls_list = []
    for i in team_row.index:
        foul = team_row.loc[i].LPF if (team_row.loc[i].WTeam == team) else team_row.loc[i].WPF
        fouls_list.append(foul)
    team_ll_games = LL_games[(LL_games.Season==season)&((LL_games.WTeam==team)|(LL_games.LTeam==team))]
    home_city = team_ll_games['City_State'].value_counts().idxmax()
    team_ll = cities_ll[(cities_ll.City == home_city.split(',')[0])& (cities_ll.State== home_city.split(',')[1])]['LL']
    distance = []
    for i in range(len(team_ll_games)):
        game = team_ll_games.iloc[i]
        distance.append(2* haversine1(team_ll.values[0][1],team_ll.values[0][0], game['LL'][1], game['LL'][0]))
    reg_seas_beatup.loc[index] = [ot,rest, np.median(fouls_list),np.median(distance), result]
reg_seas_beatup = reg_seas_beatup.dropna().astype(float)

# Creating the feature_importance values for each variable 
scores = []
ot_fe = []
dor_fe = []
mfa_fe = []
mtd_fe = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(reg_seas_beatup[['AVG OT','AVG Days of Rest','Median Fouls Against','Median Travel Distance']], reg_seas_beatup.Result, test_size=0.33)
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)
    ot, dor, mfa, mtd = rfc.feature_importances_
    ot_fe.append(ot)
    dor_fe.append(dor)
    mfa_fe.append(mfa)
    mtd_fe.append(mtd)

# normalize features to get same scale
reg_seas_beatup[['AVG OT','AVG Days of Rest','Median Fouls Against', "Median Travel Distance"]] = reg_seas_beatup[['AVG OT','AVG Days of Rest','Median Fouls Against','Median Travel Distance']].apply(lambda x: x/x.sum())

# function that creates the beatup Metric 
def create_feature(row):
    ot, dor, mfa, mtd = [np.mean(ot_fe),np.mean(dor_fe),np.mean(mfa_fe),np.mean(mtd_fe)]
    return (row['AVG OT']*ot) + (row['AVG Days of Rest']*dor) + (row['Median Fouls Against']*mfa) + (row['Median Travel Distance']*mtd)

reg_seas_beatup['Beatup'] = reg_seas_beatup.apply(create_feature,axis=1)
beatup = reg_seas_beatup[['Beatup','Result']]
beatup.reset_index(inplace=True)

In [0]:
beatup

,Team,Season,Beatup
0,Abilene Chr,2014,0.000401
1,Abilene Chr,2015,0.000521
2,Abilene Chr,2016,0.000604
3,Abilene Chr,2017,0.000516
4,Air Force,2010,0.000242
5,Air Force,2011,0.000291
6,Air Force,2012,0.000471
7,Air Force,2013,0.000314
8,Air Force,2014,0.000433
9,Air Force,2015,0.000298
